The goal of this competition is to predict a Windows machine’s probability of getting infected by various families of malware, based on different properties of that machine. The telemetry data containing these properties and the machine infections was generated by combining heartbeat and threat reports collected by Microsoft's endpoint protection solution, Windows Defender.

Each row in this dataset corresponds to a machine, uniquely identified by a MachineIdentifier. HasDetections is the ground truth and indicates that Malware was detected on the machine. Using the information and labels in train.csv, you must predict the value for HasDetections for each machine in test.csv.

The sampling methodology used to create this dataset was designed to meet certain business constraints, both in regards to user privacy as well as the time period during which the machine was running. Malware detection is inherently a time-series problem, but it is made complicated by the introduction of new machines, machines that come online and offline, machines that receive patches, machines that receive new operating systems, etc. While the dataset provided here has been roughly split by time, the complications and sampling requirements mentioned above may mean you may see imperfect agreement between your cross validation, public, and private scores! Additionally, this dataset is not representative of Microsoft customers’ machines in the wild; it has been sampled to include a much larger proportion of malware machines.

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.experimental import enable_hist_gradient_boosting
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
# set up display area to show dataframe in jupyter qtconsole
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# We need to explicitly specify data types when reading csv, otherwise it is very memory consuming
# and we will get the warning "Specify dtype option on import or set low_memory=False"
# So, we will manually defined the data types

# P.S. I have loaded the sample data and exported train_data.dtypes
# these are the data types for fast loading

datatypes = {
    'ProductName': str,
    'EngineVersion': str,
    'AppVersion': str,
    'AvSigVersion': str,
    'IsBeta': np.int8,
    'RtpStateBitfield': str,
    'IsSxsPassiveMode': np.int8,
    'DefaultBrowsersIdentifier': str,
    'AVProductStatesIdentifier': str,
    'AVProductsInstalled': str,
    'AVProductsEnabled': str,
    'HasTpm': np.int8,
    'CountryIdentifier': str,
    'CityIdentifier': str,
    'OrganizationIdentifier': str,
    'GeoNameIdentifier': str,
    'LocaleEnglishNameIdentifier': str,
    'Platform': str,
    'Processor': str,
    'OsVer': str,
    'OsBuild': str,
    'OsSuite': str,
    'OsPlatformSubRelease': str,
    'OsBuildLab': str,
    'SkuEdition': str,
    'IsProtected': str,
    'AutoSampleOptIn': np.int8,
    'PuaMode': str,
    'SMode': str,
    'IeVerIdentifier': str,
    'SmartScreen': str,
    'Firewall': str,
    'UacLuaenable': str,
    'Census_MDC2FormFactor': str,
    'Census_DeviceFamily': str,
    'Census_OEMNameIdentifier': str,
    'Census_OEMModelIdentifier': str, 
    'Census_ProcessorCoreCount': str,
    'Census_ProcessorManufacturerIdentifier': str,
    'Census_ProcessorModelIdentifier': str,
    'Census_ProcessorClass': str,
    'Census_PrimaryDiskTotalCapacity': np.float64,
    'Census_PrimaryDiskTypeName': str,
    'Census_SystemVolumeTotalCapacity': np.float64,
    'Census_HasOpticalDiskDrive': np.int8,
    'Census_TotalPhysicalRAM': np.float64,
    'Census_ChassisTypeName': str,
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': str,
    'Census_InternalPrimaryDisplayResolutionHorizontal': str,
    'Census_InternalPrimaryDisplayResolutionVertical': str,
    'Census_PowerPlatformRoleName': str,
    'Census_InternalBatteryType': str,
    'Census_InternalBatteryNumberOfCharges': str,
    'Census_OSVersion': str,
    'Census_OSArchitecture': str,
    'Census_OSBranch': str,
    'Census_OSBuildNumber': str,
    'Census_OSBuildRevision': str,
    'Census_OSEdition': str,
    'Census_OSSkuName': str,
    'Census_OSInstallTypeName': str,
    'Census_OSInstallLanguageIdentifier': str,
    'Census_OSUILocaleIdentifier': str,
    'Census_OSWUAutoUpdateOptionsName': str,
    'Census_IsPortableOperatingSystem': np.int8,
    'Census_GenuineStateName': str,
    'Census_ActivationChannel': str,
    'Census_IsFlightingInternal': str,
    'Census_IsFlightsDisabled': str,
    'Census_FlightRing': str,
    'Census_ThresholdOptIn': str,
    'Census_FirmwareManufacturerIdentifier': str,
    'Census_FirmwareVersionIdentifier': str,
    'Census_IsSecureBootEnabled': np.int8,
    'Census_IsWIMBootEnabled': str,
    'Census_IsVirtualDevice': str,
    'Census_IsTouchEnabled': np.int8,
    'Census_IsPenCapable': np.int8,
    'Census_IsAlwaysOnAlwaysConnectedCapable': str,
    'Wdft_IsGamer': str,
    'Wdft_RegionIdentifier': str,
    'HasDetections': np.int8
}

full_features = pd.read_csv("./csv/train.csv", dtype=datatypes, index_col="MachineIdentifier")
#full_features = pd.read_csv("./csv/train.csv", dtype=datatypes, nrows=200000, index_col="MachineIdentifier")

In [4]:
print (full_features.shape)

(8921483, 82)


In [5]:
# Checking the columns with the most NULL values
print((full_features.isnull().sum()).sort_values(ascending=False).head(20))

PuaMode                                  8919174
Census_ProcessorClass                    8884852
DefaultBrowsersIdentifier                8488045
Census_IsFlightingInternal               7408759
Census_InternalBatteryType               6338429
Census_ThresholdOptIn                    5667325
Census_IsWIMBootEnabled                  5659703
SmartScreen                              3177011
OrganizationIdentifier                   2751518
SMode                                     537759
CityIdentifier                            325409
Wdft_IsGamer                              303451
Wdft_RegionIdentifier                     303451
Census_InternalBatteryNumberOfCharges     268755
Census_FirmwareManufacturerIdentifier     183257
Census_IsFlightsDisabled                  160523
Census_FirmwareVersionIdentifier          160133
Census_OEMModelIdentifier                 102233
Census_OEMNameIdentifier                   95478
Firewall                                   91350
dtype: int64


In [6]:
full_features['PuaMode'].unique()

array([nan, 'on', 'audit'], dtype=object)

In [7]:
full_features['Census_IsFlightingInternal'].unique()

array([nan, '0', '1'], dtype=object)

In [8]:
full_features['Census_InternalBatteryType'].unique()

array([nan, 'lion', 'li-i', '#', 'lip', 'liio', 'vbox', 'li p', 'real',
       'unkn', 'pbac', 'li', 'bq20', 'nimh', '\x04lio', 'lgi0', 'lhp0',
       'ithi', 'batt', 'lipp', 'lipo', '4cel', 'ram', 'lit', 'a140',
       'bad', 'asmb', 'virt', 'ca48', '4ion', 'd', 'a132', 'ÿÿÿÿ', 'cl53',
       'lio', 'li-l', '÷ÿóö', 'í\x03-i', '0x0b', 'lgs0', '3ion', 'ots0',
       'lai0', 'lilo', 'pa50', 'h4°s', '5nm1', 'li-p', 'lhpo', '0ts0',
       'pad0', 'sail', 'p-sn', 'icp3', 'a130', '2337', '\x1f˙˙˙', 'lgl0',
       'l\x15', '@i\uf8f5\uf8f5', 'li\x90o', '4lio', 'lp', 'li?',
       '\x04ion', 'pbso', 'a138', 'li-h', '6ion', '3500', 'h00j',
       'li\x10', 'sams', '\x03ip', '8', '#TAB#', 'l\x06&#TAB#', 'liÿÿ',
       'lÿÿÿ'], dtype=object)

In [9]:
full_features['Census_ThresholdOptIn'].unique()

array([nan, '0', '1'], dtype=object)

In [10]:
full_features['Census_IsWIMBootEnabled'].unique()

array([nan, '0', '1'], dtype=object)

In [11]:
full_features['SMode'].unique()

array(['0', nan, '1'], dtype=object)

In [12]:
full_features['OrganizationIdentifier'].unique()

array(['18', nan, '27', '46', '11', '14', '37', '10', '50', '49', '33',
       '8', '48', '36', '31', '4', '1', '28', '3', '52', '32', '51', '5',
       '2', '47', '44', '16', '40', '20', '22', '29', '26', '21', '39',
       '6', '19', '7', '30', '42', '43', '41', '15', '45', '25', '35',
       '23', '38', '12', '17', '34'], dtype=object)

In [13]:
full_features['Wdft_IsGamer'].unique()

array(['0', '1', nan], dtype=object)

In [14]:
full_features['Wdft_RegionIdentifier'].unique()

array(['10', '8', '3', '1', '15', '7', '11', '2', '12', '4', '13', nan,
       '6', '9', '5', '14'], dtype=object)

In [15]:
full_features['CityIdentifier'].unique()

array(['128035', '1482', '153579', ..., '47472', '147921', '97837'],
      dtype=object)

In [16]:
full_features['Census_InternalBatteryNumberOfCharges'].unique()

array(['4294967295', '1', '0', ..., '27736', '26424', '16807'],
      dtype=object)

In [17]:
# Cleaning up some data

# PuaMode - Potentially Unwanted Applications, if NA, then it is disabled. 99% are NA. So, better to drop it
# Census_ProcessorClass - According to the description - "No longer maintained and updated"
# DefaultBrowsersIdentifier - Almost all values are empty. Therefore we will drop this column
# Census_IsFlightingInternal - whether this is internal or "external" testing ring. Column mostly unused. Will have to drop it
# Census_InternalBatteryType - comtains mostly garbage. Besides, it should not be relevant to attack surface.
# Census_ThresholdOptIn - also mostly unused. Googled it and Threshold was used in first versions of Windows 10. Looks like unused now
# Census_IsWIMBootEnabled - Is it possible to boot from Windows Image? Not relevant to identification of the attacks when 70% of data is emtpy
# SmartScreen - Whether smart screen in explorer is enabled. Should be important. "ExistsNotSet" when null, according to the description
# SMode - Quite relevant field. Will be keeping it
# OrganizationIdentifier - Attacks by organizations should be analyzed. If not filled, will assign "0". 
# Census_InternalBatteryNumberOfCharges - Not relevant. Will drop this column in order not to overtrain
# Census_OSSkuName -  OS edition friendly name (currently Windows only). - Can be removed. Duplicate field
# Census_ChassisTypeName - Census_MDC2FormFactor gives better information. Let's remove this field

#full_features['PuaMode'] = full_features['PuaMode'].fillna('off')
#full_features['SmartScreen'] = full_features['SmartScreen'].fillna('ExistsNotSet')
#full_features['SMode'] = full_features['SMode'].fillna('0').astype('int8')
#full_features['OrganizationIdentifier'] = full_features['OrganizationIdentifier'].fillna('0').astype('int32')
#full_features['Wdft_IsGamer'] = full_features['Wdft_IsGamer'].fillna('0').astype('int8')
#full_features['Wdft_RegionIdentifier'] = full_features['Wdft_RegionIdentifier'].fillna('0').astype('int32')
#full_features['CityIdentifier'] = full_features['CityIdentifier'].fillna('0').astype('int32')

#full_features = full_features.drop([
#    'PuaMode',
#    'Census_OSEdition',
#    'Census_ProcessorClass',
#    'DefaultBrowsersIdentifier',
#    'Census_IsFlightingInternal',
#    'Census_InternalBatteryType'], axis=1)

In [18]:
# Now let us check the string columns

string_columns = []

for colname in full_features.dtypes.keys():
    if full_features[colname].dtypes.name == "object":
        string_columns.append(colname)
        
string_columns

['ProductName',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'RtpStateBitfield',
 'DefaultBrowsersIdentifier',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsVer',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'PuaMode',
 'SMode',
 'IeVerIdentifier',
 'SmartScreen',
 'Firewall',
 'UacLuaenable',
 'Census_MDC2FormFactor',
 'Census_DeviceFamily',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_ProcessorClass',
 'Census_PrimaryDiskTypeName',
 'Census_ChassisTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'C

In [19]:
full_features[string_columns].head(10)

,ProductName,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,7,NaN,53447,1,1,29,128035,18,35,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,NaN,0,137,NaN,1,1,Desktop,Windows.Desktop,2668,9124,4,5,2341,NaN,HDD,Desktop,18.9,1440,900,Desktop,NaN,4294967295,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26,119,UNKNOWN,IS_GENUINE,Retail,NaN,0,Retail,NaN,628,36144,NaN,0,0,0,10
000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,7,NaN,53447,1,1,93,1482,18,119,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,NaN,0,137,NaN,1,1,Notebook,Windows.Desktop,2668,91656,4,5,2405,NaN,HDD,Notebook,13.9,1366,768,Mobile,NaN,1,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8,31,UNKNOWN,OFFLINE,Retail,NaN,0,NOT_SET,NaN,628,57858,NaN,0,0,0,8
000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,7,NaN,53447,1,1,86,153579,18,64,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1,NaN,0,137,RequireAdmin,1,1,Desktop,Windows.Desktop,4909,317701,4,5,1972,NaN,SSD,Desktop,21.5,1920,1080,Desktop,NaN,4294967295,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7,30,FullAuto,IS_GENUINE,OEM:NONSLP,NaN,0,Retail,NaN,142,52682,NaN,0,0,0,3
00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,7,NaN,53447,1,1,88,20710,NaN,117,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1,NaN,0,137,ExistsNotSet,1,1,Desktop,Windows.Desktop,1443,275890,4,5,2273,NaN,UNKNOWN,MiniTower,18.5,1366,768,Desktop,NaN,4294967295,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17,64,FullAuto,IS_GENUINE,OEM:NONSLP,NaN,0,Retail,NaN,355,20050,NaN,0,0,0,3
000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,7,NaN,53447,1,1,18,37376,NaN,277,75,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1,NaN,0,137,RequireAdmin,1,1,Notebook,Windows.Desktop,1443,331929,4,5,2500,NaN,HDD,Portable,14,1366,768,Mobile,lion,0,10.0.17134.191,amd64,rs4_release,17134,191,Core,CORE,Update,8,31,FullAuto,IS_GENUINE,Retail,0,0,Retail,0,355,19844,0,0,0,0,1
000016191b897145d069102325cab760,win8defender,1.1.15100.1,4.18

At first glance at the data, it becomes obvious, that the stings are either classifiers, or versions that contain 4 classifiers in them. So. in order to use the algorithms that support only numeric values we will convert classifiers like "ProductName" to integer range and the fields like AppVersion

In [20]:
def df_replacevalues(df, colname, oldvalues, newvalues, topvalue):
    # First, we need to get the most frequent value of the column
    #topvalue = df[colname].value_counts().idxmax() # Decided yo specify explicitly, so commenting out
    
    # Replace NaN values with the popular value
    df[colname].fillna(topvalue, inplace=True)
    
    # We need to make sure no other value than oldvalues exists
    indexes = df[~df[colname].isin(oldvalues)].index
    
    # If the "Garbage" values are more than 1%, then raise an error
    if len(indexes) > len(df) / 100:
        raise Exception("Not all neccessary values are present in oldvalues array")
    
    # Replace "Garbage" with the top value
    df.loc[indexes,[colname]] = topvalue
    
    print ("Previous values", df[colname].unique())
    df[colname] = pd.to_numeric(df[colname].replace(oldvalues, newvalues), errors='raise', downcast='integer')
    print ("New values", df[colname].unique())
    
#full_features["Platform"].unique()
#full_features["Platform"].value_counts()
#full_features[~full_features["ProductName"].isin(['win8defender', 'mse'])].index

In [21]:
print(full_features["ProductName"].value_counts())

colname = "ProductName"
oldvalues = ['win8defender','mse','mseprerelease']
newvalues = [1,2,2]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'win8defender')

win8defender     8826520
mse                94873
mseprerelease         53
scep                  22
windowsintune          8
fep                    7
Name: ProductName, dtype: int64
Previous values ['win8defender' 'mse' 'mseprerelease']
New values [1 2]


In [22]:
print(full_features["Platform"].value_counts())

colname = "Platform"
oldvalues = ['windows10','windows7','windows8','windows2016','Undefined']
newvalues = [10,7,8,2016,-1]

df_replacevalues(full_features, colname, oldvalues, newvalues,'Undefined')

windows10      8618715
windows8        194508
windows7         93889
windows2016      14371
Name: Platform, dtype: int64
Previous values ['windows10' 'windows7' 'windows8' 'windows2016']
New values [  10    7    8 2016]


In [23]:
print(full_features["Processor"].value_counts())

colname = "Processor"
oldvalues = ['x64','arm64','x86']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'x64')

x64      8105435
x86       815702
arm64        346
Name: Processor, dtype: int64
Previous values ['x64' 'arm64' 'x86']
New values [1 2 3]


In [24]:
colname = "OsPlatformSubRelease"

print(full_features[colname].value_counts())

oldvalues = ['rs4','rs3','rs2','rs1','windows7','windows8.1','th1','th2','prers5','Unknown']
newvalues = [504,503,502,501,        407,408,                201,202,     505,     0]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Unknown')

rs4           3915526
rs3           2503681
rs2            780270
rs1            730819
th2            411606
th1            270192
windows8.1     194508
windows7        93889
prers5          20992
Name: OsPlatformSubRelease, dtype: int64
Previous values ['rs4' 'rs1' 'rs3' 'windows7' 'windows8.1' 'th1' 'rs2' 'th2' 'prers5']
New values [504 501 503 407 408 201 502 202 505]


In [25]:
colname = "SkuEdition"

print(full_features[colname].value_counts())

oldvalues = ['Pro','Home','Invalid','Enterprise LTSB','Enterprise','Education','Cloud','Server']
newvalues = [55,52,0,71,70,20,90,80]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Invalid')

Home               5514341
Pro                3224164
Invalid              78054
Education            40694
Enterprise           34357
Enterprise LTSB      20702
Cloud                 5589
Server                3582
Name: SkuEdition, dtype: int64
Previous values ['Pro' 'Home' 'Invalid' 'Enterprise LTSB' 'Enterprise' 'Education' 'Cloud'
 'Server']
New values [55 52  0 71 70 20 90 80]


In [26]:
colname = "SmartScreen"

print(full_features[colname].value_counts())

oldvalues = ['Off','off','OFF','On','on','Warn','Prompt','ExistsNotSet','Block','RequireAdmin']
newvalues = [0,0,0,1,1,2,3,4,5,6]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'ExistsNotSet')

RequireAdmin    4316183
ExistsNotSet    1046183
Off              186553
Warn             135483
Prompt            34533
Block             22533
off                1350
On                  731
&#x02;              416
&#x01;              335
on                  147
requireadmin         10
OFF                   4
0                     3
Promt                 2
prompt                1
warn                  1
Enabled               1
&#x03;                1
00000000              1
requireAdmin          1
Name: SmartScreen, dtype: int64
Previous values ['ExistsNotSet' 'RequireAdmin' 'Off' 'Warn' 'Prompt' 'Block' 'off' 'On'
 'on' 'OFF']
New values [4 6 0 2 3 5 1]


In [27]:
colname = "Census_MDC2FormFactor"

print(full_features[colname].value_counts())

oldvalues = ['Desktop','Notebook','Detachable','PCOther','AllInOne','Convertible','SmallTablet','LargeTablet','SmallServer','LargeServer','MediumServer','ServerOther','Other']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Other')

Notebook        5723319
Desktop         1951086
Convertible      405378
Detachable       298233
AllInOne         292077
PCOther          139955
LargeTablet       67121
SmallTablet       31393
SmallServer        8630
MediumServer       3385
LargeServer         875
ServerOther          30
IoTOther              1
Name: Census_MDC2FormFactor, dtype: int64
Previous values ['Desktop' 'Notebook' 'Detachable' 'PCOther' 'AllInOne' 'Convertible'
 'SmallTablet' 'LargeTablet' 'SmallServer' 'LargeServer' 'MediumServer'
 'ServerOther' 'Other']
New values [ 1  2  3  4  5  6  7  8  9 10 11 12 13]


In [28]:
# Census_DeviceFamily ['Windows.Desktop' 'Windows.Server' 'Windows']

colname = "Census_DeviceFamily"

print(full_features[colname].value_counts())

oldvalues = ['Windows.Desktop','Windows.Server','Windows']
#newvalues = [i+1 for i in range(len(oldvalues))]
# Windows = Windows.Desktop
newvalues = [1,2,1]
    
df_replacevalues(full_features, colname, oldvalues, newvalues, 'Windows.Desktop')

Windows.Desktop    8907053
Windows.Server       14410
Windows                 20
Name: Census_DeviceFamily, dtype: int64
Previous values ['Windows.Desktop' 'Windows.Server' 'Windows']
New values [1 2]


In [29]:
# Census_PrimaryDiskTypeName ['HDD' 'SSD' 'UNKNOWN' 'Unspecified' nan]

colname = "Census_PrimaryDiskTypeName"

print(full_features[colname].value_counts())

oldvalues = ['HDD','SSD','UNKNOWN','Unspecified']
newvalues = [1,2,3,3]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Unspecified')

HDD            5806804
SSD            2466808
UNKNOWN         358251
Unspecified     276776
Name: Census_PrimaryDiskTypeName, dtype: int64
Previous values ['HDD' 'SSD' 'UNKNOWN' 'Unspecified']
New values [1 2 3]


In [30]:
# Census_ChassisTypeName Index(['Notebook', 'Desktop', 'Laptop', 'Portable', 'AllinOne', 'MiniTower', 'Convertible', 'Other', 'UNKNOWN', 'Detachable', 'LowProfileDesktop', 'HandHeld', 'SpaceSaving', 'Tablet', 'Tower', 'Unknown', 'MainServerChassis', 'MiniPC', 'LunchBox', 'RackMountChassis', 'SubNotebook', 'BusExpansionChassis', '30', 'StickPC', '0', 'MultisystemChassis', 'Blade', '35', 'PizzaBox', 'SealedCasePC', 'SubChassis', 'ExpansionChassis', '31', '32', '88', '127', '25', '44', '36', 'DockingStation', 'BladeEnclosure', 'CompactPCI', '81', '45', 'EmbeddedPC', '28', '82', '112', 'IoTGateway', '49', '76', '39'], dtype='object')

colname = "Census_ChassisTypeName"

print(full_features[colname].value_counts())

oldvalues = ['Notebook', 'Desktop', 'Laptop', 'Portable', 'AllinOne', 'MiniTower', 'Convertible', 'Other', 'UNKNOWN', 'Detachable', 
             'LowProfileDesktop', 'HandHeld', 'SpaceSaving', 'Tablet', 'Tower', 'Unknown', 'MainServerChassis', 'MiniPC', 'LunchBox', 
             'RackMountChassis', 'SubNotebook', 'BusExpansionChassis']
# Grouping Laptop/Notebook, unknown and other
newvalues = [1,2,1,3,4,5,6,0,0,7,
             8,9,10,11,12,0,13,14,15,
             16,1,17]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'UNKNOWN')

Notebook               5248812
Desktop                1872125
Laptop                  685581
Portable                360903
AllinOne                204295
MiniTower                85127
Convertible              84472
Other                    75782
UNKNOWN                  67212
Detachable               51466
LowProfileDesktop        50072
HandHeld                 46009
SpaceSaving              29070
Tablet                   13630
Tower                    12549
Unknown                  10011
MainServerChassis         9545
MiniPC                    4433
LunchBox                  3971
RackMountChassis          3410
SubNotebook                807
BusExpansionChassis        720
30                         243
StickPC                    142
0                          133
MultisystemChassis          61
Blade                       52
35                          50
PizzaBox                    46
SealedCasePC                39
SubChassis                  16
ExpansionChassis            12
31      

In [31]:
# Census_PowerPlatformRoleName Index(['Mobile', 'Desktop', 'Slate', 'Workstation', 'SOHOServer', 'UNKNOWN', 'EnterpriseServer', 'AppliancePC', 'PerformanceServer', 'Unspecified']

colname = "Census_PowerPlatformRoleName"

print(full_features[colname].value_counts())

oldvalues = ['Mobile', 'Desktop', 'Slate', 'Workstation', 'SOHOServer', 'UNKNOWN', 'EnterpriseServer', 'AppliancePC', 'PerformanceServer', 'Unspecified']
newvalues = [1,2,3,4,5,0,6,7,8,0]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'UNKNOWN')

Mobile               6182908
Desktop              2066620
Slate                 492537
Workstation           109683
SOHOServer             37841
UNKNOWN                20628
EnterpriseServer        7094
AppliancePC             4015
PerformanceServer         97
Unspecified                5
Name: Census_PowerPlatformRoleName, dtype: int64
Previous values ['Desktop' 'Mobile' 'Slate' 'Workstation' 'SOHOServer' 'UNKNOWN'
 'AppliancePC' 'EnterpriseServer' 'PerformanceServer' 'Unspecified']
New values [2 1 3 4 5 0 7 6 8]


In [32]:
# Census_OSArchitecture Index(['amd64', 'x86', 'arm64'], dtype='object')

colname = "Census_OSArchitecture"

print(full_features[colname].value_counts())

oldvalues = ['amd64', 'x86', 'arm64']
newvalues = [1,3,2]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'amd64')

amd64    8105885
x86       815252
arm64        346
Name: Census_OSArchitecture, dtype: int64
Previous values ['amd64' 'arm64' 'x86']
New values [1 2 3]


In [33]:
# Census_OSBranch Index(['rs4_release', 'rs3_release', 'rs3_release_svc_escrow', 'rs2_release', 'rs1_release', 'th2_release', 'th2_release_sec', 'th1_st1', 'th1', 'rs5_release', 'rs3_release_svc_escrow_im', 'rs_prerelease', 'rs_prerelease_flt', 'rs5_release_sigma', 'rs1_release_srvmedia', 'winblue_ltsb_escrow', 'win7sp1_ldr', 'winblue_ltsb', 'win8_gdr', 'rs_xbox', 'rs5_release_edge', 'rs5_release_sigma_dev', 'win7sp1_ldr_escrow', 'rs1_release_sec', 'rs_shell', 'rs1_release_svc', 'win8_ldr', 'rs_onecore_base_cobalt', 'rs_onecore_stack_per1', 'rs5_release_sign', 'rs3_release_svc', 'Khmer OS'], dtype='object')

colname = "Census_OSBranch"

print(full_features[colname].value_counts())

oldvalues = ['rs5_release', 'rs5_release_sigma', 'rs4_release', 'rs3_release', 'rs3_release_svc_escrow', 
             'rs3_release_svc_escrow_im', 'rs2_release', 'rs1_release', 'rs_prerelease', 'rs_prerelease_flt', 
             'th2_release', 'th2_release_sec', 'th1_st1', 'th1', 'Undefined']
newvalues = [25,25,24,23,23,23,22,21,20,20,
             12,12,11,11,0]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Undefined')

rs4_release                  4009158
rs3_release                  1237321
rs3_release_svc_escrow       1199767
rs2_release                   797066
rs1_release                   785534
th2_release                   326655
th2_release_sec               266882
th1_st1                       195840
th1                            75764
rs5_release                    15324
rs3_release_svc_escrow_im       6181
rs_prerelease                   3171
rs_prerelease_flt               2714
rs5_release_sigma                 62
rs1_release_srvmedia              10
winblue_ltsb_escrow                8
win8_gdr                           3
winblue_ltsb                       3
win7sp1_ldr                        3
rs5_release_sigma_dev              2
rs5_release_edge                   2
rs_xbox                            2
win7sp1_ldr_escrow                 2
rs_onecore_base_cobalt             1
rs_onecore_stack_per1              1
rs5_release_sign                   1
rs1_release_sec                    1
r

In [34]:
# Census_OSSkuName Index(['CORE', 'PROFESSIONAL', 'CORE_SINGLELANGUAGE', 'CORE_COUNTRYSPECIFIC', 'EDUCATION', 'ENTERPRISE', 'PROFESSIONAL_N', 'ENTERPRISE_S', 'STANDARD_SERVER', 'CLOUD', 'CORE_N', 'STANDARD_EVALUATION_SERVER', 'EDUCATION_N', 'ENTERPRISE_S_N', 'DATACENTER_EVALUATION_SERVER', 'SB_SOLUTION_SERVER', 'ENTERPRISE_N', 'PRO_WORKSTATION', 'UNLICENSED', 'DATACENTER_SERVER', 'PRO_WORKSTATION_N', 'CLOUDN', 'PRO_CHINA', 'SERVERRDSH', 'ULTIMATE', 'PRO_FOR_EDUCATION', 'PRO_SINGLE_LANGUAGE', 'UNDEFINED', 'STARTER', 'ENTERPRISEG'], dtype='object')

colname = "Census_OSSkuName"
oldvalues = ['CORE', 'CORE_SINGLELANGUAGE', 'CORE_COUNTRYSPECIFIC', 'CORE_N',
             'EDUCATION', 'EDUCATION_N',
             'PROFESSIONAL', 'PROFESSIONAL_N', 'PRO_WORKSTATION',
             'ENTERPRISE',  'ENTERPRISE_S', 'ENTERPRISE_S_N', 'ENTERPRISE_N', 
             'CLOUD',
             'SB_SOLUTION_SERVER', 'STANDARD_SERVER', 'STANDARD_EVALUATION_SERVER', 'DATACENTER_EVALUATION_SERVER', 'UNLICENSED']
newvalues = [i+1 for i in range(len(oldvalues))]

# Group this feature by values

full_features['CORE'] = 1 if 'CORE' in full_features['Census_OSSkuName'] else 0
full_features['EDUCATION'] = 1 if 'EDUCATION' in full_features['Census_OSSkuName'] else 0
full_features['PRO'] = 1 if 'PRO' in full_features['Census_OSSkuName'] else 0
full_features['ENTERPRISE'] = 1 if 'ENTERPRISE' in full_features['Census_OSSkuName'] else 0
full_features['CLOUD'] = 1 if 'CLOUD' in full_features['Census_OSSkuName'] else 0
full_features['SERVER'] = 1 if 'SERVER' in full_features['Census_OSSkuName'] else 0
full_features['EVALUATION'] = 1 if 'EVALUATION' in full_features['Census_OSSkuName'] else 0

full_features.drop([colname], axis=1, inplace=True)


In [35]:
# Census_OSInstallTypeName Index(['UUPUpgrade', 'IBSClean', 'Update', 'Upgrade', 'Other', 'Reset', 'Refresh', 'Clean', 'CleanPCRefresh'], dtype='object')

colname = "Census_OSInstallTypeName"

print(full_features[colname].value_counts())

oldvalues = ['UUPUpgrade', 'IBSClean', 'Update', 'Upgrade', 'Other', 'Reset', 'Refresh', 'Clean', 'CleanPCRefresh']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Other')

UUPUpgrade        2608037
IBSClean          1650733
Update            1593308
Upgrade           1251559
Other              840121
Reset              649201
Refresh            205842
Clean               69073
CleanPCRefresh      53609
Name: Census_OSInstallTypeName, dtype: int64
Previous values ['UUPUpgrade' 'IBSClean' 'Update' 'Upgrade' 'Other' 'Clean' 'Reset'
 'Refresh' 'CleanPCRefresh']
New values [1 2 3 4 5 8 6 7 9]


In [36]:
# Census_OSWUAutoUpdateOptionsName Index(['FullAuto', 'UNKNOWN', 'Notify', 'AutoInstallAndRebootAtMaintenanceTime', 'Off', 'DownloadNotify'], dtype='object')

colname = "Census_OSWUAutoUpdateOptionsName"

print(full_features[colname].value_counts())

oldvalues = ['FullAuto', 'UNKNOWN', 'Notify', 'AutoInstallAndRebootAtMaintenanceTime', 'Off', 'DownloadNotify']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'UNKNOWN')

FullAuto                                 3954497
UNKNOWN                                  2519925
Notify                                   2034254
AutoInstallAndRebootAtMaintenanceTime     371475
Off                                        26961
DownloadNotify                             14371
Name: Census_OSWUAutoUpdateOptionsName, dtype: int64
Previous values ['UNKNOWN' 'FullAuto' 'Notify' 'AutoInstallAndRebootAtMaintenanceTime'
 'Off' 'DownloadNotify']
New values [2 1 3 4 5 6]


In [37]:
colname = "Census_InternalBatteryType"

print(full_features[colname].value_counts())

oldvalues = ['lion', 'li-i', '#', 'lip', 'unkn']
newvalues = [1,1,1,1,2]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'unkn')

lion        2028256
li-i         245617
#            183998
lip           62099
liio          32635
li p           8383
li             6708
nimh           4614
real           2744
bq20           2302
pbac           2274
vbox           1454
unkn            533
lgi0            399
lipo            198
lhp0            182
4cel            170
lipp             83
ithi             79
batt             60
ram              35
bad              33
virt             33
pad0             22
ca48             16
lit              16
a132             10
ots0              9
ÿÿÿÿ              8
lai0              8
lio               5
asmb              4
lio              4
li-p              4
4lio              4
icp3              3
0x0b              3
lgs0              3
3ion              2
h00j              2
a138              2
a140              2
lhpo              2
5nm1              2
sail              1
li-l              1
pa50              1
li               1
lio              1
8                 1


In [38]:
# Census_GenuineStateName Index(['IS_GENUINE', 'INVALID_LICENSE', 'OFFLINE', 'UNKNOWN', 'TAMPERED'], dtype='object')

colname = "Census_GenuineStateName"

print(full_features[colname].value_counts())

oldvalues = ['IS_GENUINE', 'INVALID_LICENSE', 'OFFLINE', 'UNKNOWN', 'TAMPERED']
newvalues = [1,2,3,4,2]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'UNKNOWN')

IS_GENUINE         7877597
INVALID_LICENSE     801692
OFFLINE             228366
UNKNOWN              13826
TAMPERED                 2
Name: Census_GenuineStateName, dtype: int64
Previous values ['IS_GENUINE' 'OFFLINE' 'INVALID_LICENSE' 'UNKNOWN' 'TAMPERED']
New values [1 3 2 4]


In [39]:
# Census_ActivationChannel Index(['Retail', 'OEM:DM', 'Volume:GVLK', 'OEM:NONSLP', 'Volume:MAK', 'Retail:TB:Eval'], dtype='object')

#Assigning separate values for Retail, OEM and Volume channels
colname = "Census_ActivationChannel"

print(full_features[colname].value_counts())

oldvalues = ['Retail', 'Retail:TB:Eval', 'OEM:DM', 'OEM:NONSLP', 'Volume:GVLK', 'Volume:MAK', 'Other']
#newvalues = [i+1 for i in range(len(oldvalues))]
newvalues = [1,1,2,2,3,3,4]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Other')

Retail            4727589
OEM:DM            3413350
Volume:GVLK        450954
OEM:NONSLP         317980
Volume:MAK           8028
Retail:TB:Eval       3582
Name: Census_ActivationChannel, dtype: int64
Previous values ['Retail' 'OEM:NONSLP' 'OEM:DM' 'Volume:GVLK' 'Volume:MAK'
 'Retail:TB:Eval']
New values [1 2 3]


In [40]:
full_features['Census_FlightRing'].value_counts()

Retail      8355679
NOT_SET      287803
Unknown      243438
WIS           10648
WIF           10322
RP             9860
Disabled       3722
OSG               7
Canary            3
Invalid           1
Name: Census_FlightRing, dtype: int64

In [41]:
# Census_FlightRing Index(['Retail', 'NOT_SET', 'Unknown', 'WIS', 'WIF', 'RP', 'Disabled', 'OSG', 'Canary', 'Invalid', 'CBCanary'], dtype='object')

colname = "Census_FlightRing"

print(full_features[colname].value_counts())

oldvalues = ['Retail', 'NOT_SET', 'Disabled', 'Unknown']
newvalues = [1,2,2,3]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Unknown')

Retail      8355679
NOT_SET      287803
Unknown      243438
WIS           10648
WIF           10322
RP             9860
Disabled       3722
OSG               7
Canary            3
Invalid           1
Name: Census_FlightRing, dtype: int64
Previous values ['Retail' 'NOT_SET' 'Unknown' 'Disabled']
New values [1 2 3]


In [42]:
# Census_FlightRing Index(['Retail', 'NOT_SET', 'Unknown', 'WIS', 'WIF', 'RP', 'Disabled', 'OSG', 'Canary', 'Invalid', 'CBCanary'], dtype='object')

colname = "Census_OSEdition"

print(full_features[colname].value_counts())

oldvalues = ['Core','CoreSingleLanguage','CoreCountrySpecific','CoreN',
             'Professional','ProfessionalN','ProfessionalEducation','ProfessionalEducationN',
             'Education','EducationN',
             'Enterprise','EnterpriseS','EnterpriseSN','EnterpriseN',
             'ServerStandard','ServerStandardEval','ServerDatacenterEval','ServerSolution',
             'Cloud',
             'Other']
newvalues = [1,1,1,1,
             2,2,2,2,
             3,3,
             4,4,4,4,
             5,5,5,5,
             6,
             7]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'Other')

Core                           3469991
Professional                   3130566
CoreSingleLanguage             1945461
CoreCountrySpecific             166100
ProfessionalEducation            56698
Education                        40704
Enterprise                       35603
ProfessionalN                    28341
EnterpriseS                      20020
ServerStandard                   10128
Cloud                             6275
CoreN                             4790
ServerStandardEval                2751
EducationN                         932
EnterpriseSN                       878
ServerDatacenterEval               829
ServerSolution                     683
EnterpriseN                        351
ProfessionalEducationN             192
ProfessionalWorkstation            128
ServerDatacenter                    15
ProfessionalWorkstationN            13
CloudN                               8
ProfessionalCountrySpecific          5
ServerRdsh                           4
Ultimate                 

In [43]:
# PuaMode Index(['off', 'on', 'audit'], dtype='object')

#colname = "PuaMode"

#print(full_features[colname].value_counts())

#oldvalues = ['off', 'on', 'audit']
#newvalues = [0,1,2]

#df_replacevalues(full_features, colname, oldvalues, newvalues)

full_features.drop(['PuaMode','Census_ProcessorClass','DefaultBrowsersIdentifier'], axis=1, inplace=True)

In [44]:
# Now let us check the string columns again

string_columns = []

for colname in full_features.dtypes.keys():
    if full_features[colname].dtypes.name == "object":
        string_columns.append(colname)
        
string_columns

['EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'RtpStateBitfield',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsVer',
 'OsBuild',
 'OsSuite',
 'OsBuildLab',
 'IsProtected',
 'SMode',
 'IeVerIdentifier',
 'Firewall',
 'UacLuaenable',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSVersion',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_IsFlightingInternal',
 'Census_IsFlightsDisabled',
 'Census_ThresholdOptIn',
 'Cens

In [45]:
full_features[string_columns].head(10)

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsVer,OsBuild,OsSuite,OsBuildLab,IsProtected,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,1.1.15100.1,4.18.1807.18075,1.273.1735.0,7,53447,1,1,29,128035,18,35,171,10.0.0.0,17134,256,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,2668,9124,4,5,2341,18.9,1440,900,4294967295,10.0.17134.165,17134,165,26,119,NaN,0,NaN,628,36144,NaN,0,0,0,10
000007535c3f730efa9ea0b7ef1bd645,1.1.14600.4,4.13.17134.1,1.263.48.0,7,53447,1,1,93,1482,18,119,64,10.0.0.0,17134,256,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,2668,91656,4,5,2405,13.9,1366,768,1,10.0.17134.1,17134,1,8,31,NaN,0,NaN,628,57858,NaN,0,0,0,8
000007905a28d863f6d0d597892cd692,1.1.15100.1,4.18.1807.18075,1.273.1341.0,7,53447,1,1,86,153579,18,64,49,10.0.0.0,17134,768,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,4909,317701,4,5,1972,21.5,1920,1080,4294967295,10.0.17134.165,17134,165,7,30,NaN,0,NaN,142,52682,NaN,0,0,0,3
00000b11598a75ea8ba1beea8459149f,1.1.15100.1,4.18.1807.18075,1.273.1527.0,7,53447,1,1,88,20710,NaN,117,115,10.0.0.0,17134,256,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,1443,275890,4,5,2273,18.5,1366,768,4294967295,10.0.17134.228,17134,228,17,64,NaN,0,NaN,355,20050,NaN,0,0,0,3
000014a5f00daa18e76b81417eeb99fc,1.1.15100.1,4.18.1807.18075,1.273.1379.0,7,53447,1,1,18,37376,NaN,277,75,10.0.0.0,17134,768,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,1443,331929,4,5,2500,14,1366,768,0,10.0.17134.191,17134,191,8,31,0,0,0,355,19844,0,0,0,0,1
000016191b897145d069102325cab760,1.1.15100.1,4.18.1807.18075,1.273.1094.0,7,53447,1,1,97,13598,27,126,124,10.0.0.0,17134,256,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,3799,340727,2,5,4324,21.5,1920,1080,4294967295,10.0.17134.165,17134,165,18,72,0,0,0,93,51039,0,0,0,0,15
0000161e8abf8d8b89c5ab8787fd712b,1.1.15100.1,4.18.1807.18075,1.273.845.0,7,43927,2,1,78,81215,NaN,89,88,10.0.0.0,17134,768,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,3799,207404,2,1,657,17.2,1600,900,0,10.0.17134.165,17134,165,14,49,NaN,0,NaN,556,63175,NaN,0,0,0,10
000019515bc8f95851aff6de873405e8,1.1.15100.1,4.18.1807.18075,1.273.1393.0,7,53447,1,1,97,150323,27,126,124,10.0.0.0,14393,768,14393.0.amd64fre.rs1_release.160715-1616,1,0,94,1,1,5682,338896,2,5,3381,15.5,1366,768,0,10.0.14393.0,14393,0,18,72,0,0,0,512,63122,0,0,0,0,15
00001a027a0ab970c408182df8484fce,1.1.15200.1,4.18.1807.18075,1.275.988.0,7,53447,1,1,164,155006,27,205,172,10.0.0.0,17134,256,17134.1.amd64fre.rs4_release.180410-1804,1,0,137,1,1,2206,240688,4,5,2836,15.6,1920,1080,0,10.0.17134.254,17134,254,27,120,NaN,0,0,500,15510,0,0,0,0,15


In [46]:
# Now we need to process the columns that contain version numbers
# We will split them in 4-5 different columns

versions = ['EngineVersion','AppVersion','AvSigVersion','OsVer','OsBuildLab','Census_OSVersion']
newcolumnnames = []

for colname in versions:
    data = full_features[colname].str.split(r"\.|-",expand=True) # Split if '.' or '-'
    for i in range(data.shape[1]):
        newcolumnname = "%s_%d" % (colname, i+1)
        newcolumnnames.append(newcolumnname)
        full_features[newcolumnname] = data[i]

In [47]:
full_features[newcolumnnames].head(10)

,EngineVersion_1,EngineVersion_2,EngineVersion_3,EngineVersion_4,AppVersion_1,AppVersion_2,AppVersion_3,AppVersion_4,AvSigVersion_1,AvSigVersion_2,AvSigVersion_3,AvSigVersion_4,OsVer_1,OsVer_2,OsVer_3,OsVer_4,OsBuildLab_1,OsBuildLab_2,OsBuildLab_3,OsBuildLab_4,OsBuildLab_5,OsBuildLab_6,Census_OSVersion_1,Census_OSVersion_2,Census_OSVersion_3,Census_OSVersion_4
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,1,1,15100,1,4,18,1807,18075,1,273,1735,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
000007535c3f730efa9ea0b7ef1bd645,1,1,14600,4,4,13,17134,1,1,263,48,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,1
000007905a28d863f6d0d597892cd692,1,1,15100,1,4,18,1807,18075,1,273,1341,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
00000b11598a75ea8ba1beea8459149f,1,1,15100,1,4,18,1807,18075,1,273,1527,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,228
000014a5f00daa18e76b81417eeb99fc,1,1,15100,1,4,18,1807,18075,1,273,1379,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,191
000016191b897145d069102325cab760,1,1,15100,1,4,18,1807,18075,1,273,1094,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
0000161e8abf8d8b89c5ab8787fd712b,1,1,15100,1,4,18,1807,18075,1,273,845,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,165
000019515bc8f95851aff6de873405e8,1,1,15100,1,4,18,1807,18075,1,273,1393,0,10,0,0,0,14393,0,amd64fre,rs1_release,160715,1616,10,0,14393,0
00001a027a0ab970c408182df8484fce,1,1,15200,1,4,18,1807,18075,1,275,988,0,10,0,0,0,17134,1,amd64fre,rs4_release,180410,1804,10,0,17134,254


In [48]:
#colname = "OsBuildLab_4"
#print (full_features[colname].value_counts())
#print (colname, full_features[colname].value_counts().keys())

In [49]:
# After splitting the columns, the only values we need to remap are OsBuildLab_3 and OsBuildLab_4
# Other values are already numeric

# OsBuildLab_3 Index(['amd64fre', 'x86fre', 'arm64fre'], dtype='object')

colname = "OsBuildLab_3"
oldvalues = ['amd64fre', 'x86fre', 'arm64fre']
newvalues = [1,3,2]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'amd64fre')

Previous values ['amd64fre' 'arm64fre' 'x86fre']
New values [1 2 3]


In [50]:
# OsBuildLab_4 Index(['rs4_release', 'rs3_release_svc_escrow', 'rs3_release', 'rs2_release', 'rs1_release', 'th2_release_sec', 'th1', 'winblue_ltsb_escrow', 'th2_release', 'rs1_release_inmarket', 'winblue_ltsb', 'win7sp1_ldr', 'rs3_release_svc', 'rs1_release_1', 'win7sp1_ldr_escrow', 'rs1_release_sec', 'th1_st1', 'rs5_release', 'rs1_release_inmarket_aim', 'rs3_release_svc_escrow_im', 'th2_release_inmarket', 'rs_prerelease', 'rs_prerelease_flt', 'win7sp1_gdr', 'winblue_gdr', 'th1_escrow', 'win7_gdr', 'winblue_r4', 'rs1_release_inmarket_rim', 'rs1_release_d', 'winblue_r9', 'winblue_r5', 'win7_rtm', 'win7sp1_rtm', 'winblue_r7', 'winblue_r3', 'winblue_r8', 'rs5_release_sigma', 'win7_ldr', 'rs5_release_sigma_dev', 'rs_xbox', 'rs5_release_edge', 'winblue_rtm', 'win7sp1_rc', 'rs3_release_svc_sec', 'rs_onecore_base_cobalt', 'rs6_prerelease', 'rs_onecore_sigma_grfx_dev', 'rs_onecore_stack_per1', 'rs5_release_sign', 'rs_shell']

colname = "OsBuildLab_4"
oldvalues = ['rs6_prerelease',
             'rs5_release', 'rs5_release_sigma', 'rs5_release_sigma_dev', 'rs5_release_edge', 'rs5_release_sign',
             'rs4_release', 
             'rs3_release_svc_escrow', 'rs3_release', 'rs3_release_svc', 'rs3_release_svc_escrow_im', 'rs3_release_svc_sec', 
             'rs2_release', 
             'rs1_release', 'rs1_release_inmarket', 'rs1_release_1', 'rs1_release_sec', 'rs1_release_inmarket_aim', 'rs1_release_inmarket_rim', 'rs1_release_d', 
             'rs_prerelease', 'rs_prerelease_flt',
             'th2_release_sec', 'th2_release', 'th2_release_inmarket', 
             'th1', 'th1_st1', 'th1_escrow', 
             'winblue_ltsb_escrow', 'winblue_ltsb', 'winblue_gdr', 'winblue_r4', 'winblue_r7', 'winblue_r3', 'winblue_r8', 'winblue_r9', 'winblue_r5', 'winblue_rtm',
             'win7sp1_ldr', 'win7sp1_ldr_escrow', 'win7sp1_gdr', 'win7_gdr', 'win7_rtm', 'win7sp1_rtm', 'win7_ldr', 'win7sp1_rc', 
             'rs_xbox', 'rs_onecore_base_cobalt', 'rs_onecore_sigma_grfx_dev', 'rs_onecore_stack_per1', 'rs_shell',
             'other']
newvalues = [i+1 for i in range(len(oldvalues))]

df_replacevalues(full_features, colname, oldvalues, newvalues, 'other')

Previous values ['rs4_release' 'rs1_release' 'rs3_release_svc_escrow' 'win7sp1_gdr'
 'rs3_release' 'winblue_ltsb_escrow' 'th1' 'rs1_release_inmarket'
 'rs2_release' 'th2_release' 'winblue_ltsb' 'th2_release_sec'
 'rs3_release_svc_escrow_im' 'th1_st1' 'th2_release_inmarket'
 'rs1_release_sec' 'rs1_release_1' 'win7sp1_ldr' 'win7sp1_ldr_escrow'
 'rs3_release_svc' 'rs1_release_inmarket_aim' 'rs5_release'
 'rs_prerelease_flt' 'rs1_release_inmarket_rim' 'winblue_r7'
 'rs_prerelease' 'win7_gdr' 'rs5_release_sigma' 'winblue_gdr' 'winblue_r4'
 'win7sp1_rtm' 'rs1_release_d' 'th1_escrow' 'rs5_release_edge'
 'winblue_r8' 'win7_ldr' 'winblue_r5' 'win7_rtm' 'other' 'winblue_r9'
 'winblue_r3' 'rs3_release_svc_sec' 'rs5_release_sigma_dev' 'win7sp1_rc'
 'rs_shell' 'rs_onecore_stack_per1' 'rs_onecore_sigma_grfx_dev'
 'winblue_rtm' 'rs_onecore_base_cobalt' 'rs_xbox' 'rs6_prerelease'
 'rs5_release_sign']
New values [ 7 14  8 41  9 29 26 15 13 24 30 23 11 27 25 17 16 39 40 10 18  2 22 19
 33 21 42  3 31 32

In [51]:
versions = ['EngineVersion','AppVersion','AvSigVersion','OsVer','OsBuildLab','Census_OSVersion']

full_features = full_features.drop(versions, axis=1)

In [52]:
for colname in full_features.columns:
    if full_features[colname].dtypes.name not in ["int8","int16","int32"]:
        #topvalue = full_features[colname].value_counts().idxmax()
        topvalue = -1
        full_features[colname].fillna(topvalue, inplace=True)
        full_features[colname] = pd.to_numeric(full_features[colname], errors='coerce')
        full_features[colname].fillna(topvalue, inplace=True)
        

In [53]:
full_features.head(10)

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections,CORE,EDUCATION,PRO,ENTERPRISE,CLOUD,SERVER,EVALUATION,EngineVersion_1,EngineVersion_2,EngineVersion_3,EngineVersion_4,AppVersion_1,AppVersion_2,AppVersion_3,AppVersion_4,AvSigVersion_1,AvSigVersion_2,AvSigVersion_3,AvSigVersion_4,OsVer_1,OsVer_2,OsVer_3,OsVer_4,OsBuildLab_1,OsBuildLab_2,OsBuildLab_3,OsBuildLab_4,OsBuildLab_5,OsBuildLab_6,Census_OSVersion_1,Census_OSVersion_2,Census_OSVersion_3,Census_OSVersion_4
MachineIdentifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000028988387b115f69f31a3bf04f09,1,0,7,0,53447,1,1,1,29,128035,18,35,171,10,1,17134,256,504,55,1,0,0,137,4,1,1,1,1,2668,9124,4,5,2341,476940.0,1,299451.0,0,4096.0,2,18.9,1440,900,2,2,4.294967e+09,1,24,17134,165,2,1,26,119,2,0,1,1,-1,0,1,-1,628,36144,0,-1,0,0,0,0,0,10,0,0,0,0,0,0,0,0,1,1,15100,1,4,18,1807,18075,1,273.0,1735,0,10,0,0,0,17134,1,1,7,180410,1804,10,0,17134,165
000007535c3f730efa9ea0b7ef1bd645,1,0,7,0,53447,1,1,1,93,1482,18,119,64,10,1,17134,256,504,55,1,0,0,137,4,1,1,2,1,2668,91656,4,5,2405,476940.0,1,102385.0,0,4096.0,1,13.9,1366,768,1,2,1.000000e+00,1,24,17134,1,2,2,8,31,2,0,3,1,-1,0,2,-1,628,57858,0,-1,0,0,0,0,0,8,0,0,0,0,0,0,0,0,1,1,14600,4,4,13,17134,1,1,263.0,48,0,10,0,0,0,17134,1,1,7,180410,1804,10,0,17134,1
000007905a28d863f6d0d597892cd692,1,0,7,0,53447,1,1,1,86,153579,18,64,49,10,1,17134,768,504,52,1,0,0,137,6,1,1,1,1,4909,317701,4,5,1972,114473.0,2,113907.0,0,4096.0,2,21.5,1920,1080,2,2,4.294967e+09,1,24,17134,165,1,1,7,30,1,0,1,2,-1,0,1,-1,142,52682,0,-1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,1,15100,1,4,18,1807,18075,1,273.0,1341,0,10,0,0,0,17134,1,1,7,180410,1804,10,0,17134,165
00000b11598a75ea8ba1beea8459149f,1,0,7,0,53447,1,1,1,88,20710,-1,117,115,10,1,17134,256,504,55,1,0,0,137,4,1,1,1,1,1443,275890,4,5,2273,238475.0,3,227116.0,0,4096.0,5,18.5,1366,768,2,2,4.294967e+09,1,24,17134,228,2,1,17,64,1,0,1,2,-1,0,1,-1,355,20050,0,-1,0,0,0,0,0,3,1,0,0,0,0,0,0,0,1,1,15100,1,4,18,1807,18075,1,273.0,1527,0,10,0,0,0,17134,1,1,7,180410,1804,10,0,17134,228
000014a5f00daa18e76b81417eeb99fc,1,0,7,0,53447,1,1,1,18,37376,-1,277,75,10,1,17134,768,504,52,1,0,0,137,6,1,1,2,1,1443,331929,4,5,2500,476940.0,1,101900.0,0,6144.0,3,14.0,1366,768,1,1,0.000000e+00,1,24,17134,191,1,3,8,31,1,0,1,1,0,0,1,0,355,19844,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,15100,1,4,18,1807,18075,1,273.0,1379,0,10,0,0,

In [54]:
# Let's see some details of the loaded data
full_features.describe()

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsBuild,OsSuite,OsPlatformSubRelease,SkuEdition,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections,CORE,EDUCATION,PRO,ENTERPRISE,CLOUD,SERVER,EVALUATION,EngineVersion_1,EngineVersion_2,EngineVersion_3,EngineVersion_4,AppVersion_1,AppVersion_2,AppVersion_3,AppVersion_4,AvSigVersion_1,AvSigVersion_2,AvSigVersion_3,AvSigVersion_4,OsVer_1,OsVer_2,OsVer_3,OsVer_4,OsBuildLab_1,OsBuildLab_2,OsBuildLab_3,OsBuildLab_4,OsBuildLab_5,OsBuildLab_6,Census_OSVersion_1,Census_OSVersion_2,Census_OSVersion_3,Census_OSVersion_4
count,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8921483.0,8921483.0,8921483.0,8921483.0,8921483.0,8921483.0,8921483.0,8921483.0,8921483.0,8.921483e+06,8.921483e+06,8921483.0,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8921483.0,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06,8.921483e+06
mean,1.010640e+00,7.509962e-06,6.816913e+00,1.733378e-02,4.764577e+04,1.317332e+00,1.012762e+00,9.879711e-01,1.080490e+02,7.830232e+04,1.688779e+01,1.696575e+02,1.228161e+02,1.315615e+01,1.182901e+00,1.571997e+04,5.751534e+02,4.770775e+02,5.263172e+01,9.377631e-01,2.891896e-05,-5.984184e-02,1.257338e+02,4.851331e+00,9.583232e-01,1.301069e+01,2.199818e+00,1.001615e+00,2.196395e+03,2.364020e+05,3.966594e+00,4.505936e+00,2.360278e+03,3.070696e+06,1.421740e+00,3.751264e+05,7.718728e-02,6.060050e+03,1.635429e+00,1.658281e+01,1.539560e+03,8.928383e+02,1.400344e+00,1.717539e+00,1.089928e+09,1.182800e

In [55]:
full_features['UacLuaenable'].unique()

array([       1,        0,       -1,       48,        3,        2,
        6357062,       49, 16777216,        5,      255,  7798884])

In [56]:
#full_features.to_csv('./csv/train_v11.csv')

In [57]:
full_features.dtypes

ProductName                                             int8
IsBeta                                                  int8
RtpStateBitfield                                       int64
IsSxsPassiveMode                                        int8
AVProductStatesIdentifier                              int64
AVProductsInstalled                                    int64
AVProductsEnabled                                      int64
HasTpm                                                  int8
CountryIdentifier                                      int64
CityIdentifier                                         int64
OrganizationIdentifier                                 int64
GeoNameIdentifier                                      int64
LocaleEnglishNameIdentifier                            int64
Platform                                               int16
Processor                                               int8
OsBuild                                                int64
OsSuite                 

In [58]:
# Shuffle the data
#np.random.seed(0)

shuffle = np.random.permutation(np.arange(full_features.shape[0]))
indexes = full_features.index[shuffle]

full_features = full_features.loc[indexes,:]

In [59]:
full_labels = full_features["HasDetections"]

# Dropping labels ["HasDetections"] from training dataset
full_features = full_features.drop(["HasDetections"], axis=1)

In [60]:
train_count = int(len(full_features) * 0.8)

train_features = full_features.values[:train_count]
test_features  = full_features.values[train_count:]

train_labels = full_labels.values[:train_count]
test_labels = full_labels.values[train_count:]

In [61]:
train_features.shape

(7137186, 104)

In [62]:
test_features.shape

(1784297, 104)

In [63]:
scaler = StandardScaler()
scaler.fit(train_features)
normalized_train_features = scaler.transform(train_features)
normalized_test_features = scaler.transform(test_features)

clf = ske.HistGradientBoostingClassifier(random_state=123)
clf.fit(normalized_train_features, train_labels)
all_columns_score = clf.score(normalized_test_features, test_labels)
    
print ("All columns (normalized)", train_features.shape, test_features.shape, train_labels.shape, test_labels.shape, "HistGradientBoostingClassifier", all_columns_score*100)

All columns (normalized) (7137186, 104) (1784297, 104) (7137186,) (1784297,) HistGradientBoostingClassifier 64.42996877761942


In [64]:
bruteforced_columns = ['ProductName', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode',
       'AVProductStatesIdentifier', 'AVProductsInstalled', 'AVProductsEnabled',
       'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition',
       'IsProtected', 'AutoSampleOptIn', 'SMode', 'IeVerIdentifier',
       'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor',
       'Census_DeviceFamily', 'Census_OEMNameIdentifier',
       'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity',
       'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM',
       'Census_ChassisTypeName',
       'Census_InternalPrimaryDiagonalDisplaySizeInInches',
       'Census_InternalPrimaryDisplayResolutionHorizontal',
       'Census_InternalPrimaryDisplayResolutionVertical',
       'Census_PowerPlatformRoleName', 'Census_InternalBatteryNumberOfCharges',
       'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber',
       'Census_OSBuildRevision', 'Census_OSEdition',
       'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier',
       'Census_OSUILocaleIdentifier', 'Census_OSWUAutoUpdateOptionsName',
       'Census_IsPortableOperatingSystem', 'Census_GenuineStateName',
       'Census_ActivationChannel', 'Census_IsFlightsDisabled',
       'Census_FlightRing', 'Census_ThresholdOptIn',
       'Census_FirmwareManufacturerIdentifier',
       'Census_FirmwareVersionIdentifier', 'Census_IsSecureBootEnabled',
       'Census_IsWIMBootEnabled', 'Census_IsVirtualDevice',
       'Census_IsTouchEnabled', 'Census_IsPenCapable',
       'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer',
       'Wdft_RegionIdentifier', 'EngineVersion_1', 'EngineVersion_2',
       'EngineVersion_3', 'EngineVersion_4', 'AppVersion_1', 'AppVersion_2',
       'AppVersion_3', 'AppVersion_4', 'AvSigVersion_1', 'AvSigVersion_2',
       'AvSigVersion_3', 'AvSigVersion_4', 'OsVer_1', 'OsVer_2', 'OsVer_3',
       'OsVer_4', 'OsBuildLab_1', 'OsBuildLab_2', 'OsBuildLab_3',
       'OsBuildLab_4', 'OsBuildLab_5', 'OsBuildLab_6', 'Census_OSVersion_1',
       'Census_OSVersion_2', 'Census_OSVersion_3', 'Census_OSVersion_4',
       'CORE', 'EDUCATION', 'PRO', 'ENTERPRISE', 'CLOUD', 'SERVER', 'EVALUATION']

bruteforced_train_features = full_features[bruteforced_columns].values[:train_count]
bruteforced_test_features  = full_features[bruteforced_columns].values[train_count:]

print ("Bruteforced", bruteforced_train_features.shape, bruteforced_test_features.shape, train_labels.shape, test_labels.shape)

Bruteforced (7137186, 98) (1784297, 98) (7137186,) (1784297,)


In [65]:
clf = ske.HistGradientBoostingClassifier(random_state=123)
clf.fit(bruteforced_train_features, train_labels)
bruteforced_columns_score = clf.score(bruteforced_test_features, test_labels)
    
print ("Bruteforced", bruteforced_train_features.shape, bruteforced_test_features.shape, 
       train_labels.shape, test_labels.shape, "HistGradientBoostingClassifier", bruteforced_columns_score*100)

Bruteforced (7137186, 98) (1784297, 98) (7137186,) (1784297,) HistGradientBoostingClassifier 64.46129764271306


In [66]:
full_features[bruteforced_columns].dtypes

ProductName                                             int8
IsBeta                                                  int8
RtpStateBitfield                                       int64
IsSxsPassiveMode                                        int8
AVProductStatesIdentifier                              int64
AVProductsInstalled                                    int64
AVProductsEnabled                                      int64
CountryIdentifier                                      int64
CityIdentifier                                         int64
OrganizationIdentifier                                 int64
GeoNameIdentifier                                      int64
LocaleEnglishNameIdentifier                            int64
Platform                                               int16
Processor                                               int8
OsSuite                                                int64
OsPlatformSubRelease                                   int16
SkuEdition              

In [67]:
engineered_columns = bruteforced_columns

full_features['ScreenProportion'] = full_features['Census_InternalPrimaryDisplayResolutionHorizontal'] / full_features['Census_InternalPrimaryDisplayResolutionVertical']
full_features['ScreenDimensions'] = (full_features['Census_InternalPrimaryDisplayResolutionHorizontal'] * 10000) + full_features['Census_InternalPrimaryDisplayResolutionVertical']
full_features['CapacityDifference'] = full_features['Census_SystemVolumeTotalCapacity'] / full_features['Census_PrimaryDiskTotalCapacity']
full_features['CapacityRatio'] = full_features['Census_SystemVolumeTotalCapacity'] - full_features['Census_PrimaryDiskTotalCapacity']
full_features['RAMByCores'] = full_features['Census_TotalPhysicalRAM'] / full_features['Census_ProcessorCoreCount'] 

full_features['ScreenProportion'] = full_features['ScreenProportion'].replace([np.inf, -np.inf], np.nan).fillna(-1)
full_features['ScreenDimensions'] = full_features['ScreenDimensions'].replace([np.inf, -np.inf], np.nan).fillna(-1)
full_features['CapacityDifference'] = full_features['CapacityDifference'].replace([np.inf, -np.inf], np.nan).fillna(-1)
full_features['CapacityRatio'] = full_features['CapacityRatio'].replace([np.inf, -np.inf], np.nan).fillna(-1)
full_features['RAMByCores'] = full_features['RAMByCores'].replace([np.inf, -np.inf], np.nan).fillna(-1)

engineered_columns.extend(['ScreenProportion', 'ScreenDimensions','CapacityDifference','CapacityRatio','RAMByCores'])

engineered_train_features = full_features[engineered_columns].values[:train_count]
engineered_test_features  = full_features[engineered_columns].values[train_count:]

print ("Engineered", engineered_train_features.shape, engineered_test_features.shape, train_labels.shape, test_labels.shape)

Engineered (7137186, 103) (1784297, 103) (7137186,) (1784297,)


In [68]:
clf = ske.HistGradientBoostingClassifier(random_state=123)
clf.fit(engineered_train_features, train_labels)
engineered_columns_score = clf.score(engineered_test_features, test_labels)
    
print ("Engineered", engineered_train_features.shape, engineered_test_features.shape, 
       train_labels.shape, test_labels.shape, "HistGradientBoostingClassifier", engineered_columns_score*100)

Engineered (7137186, 103) (1784297, 103) (7137186,) (1784297,) HistGradientBoostingClassifier 64.47783076472135


In [72]:
full_features["HasDetections"] = full_labels
engineered_columns.append("HasDetections")

In [74]:
full_features[engineered_columns].to_csv('./csv/train_v11_engineered.csv')

In [75]:
full_features[engineered_columns].dtypes

ProductName                                             int8
IsBeta                                                  int8
RtpStateBitfield                                       int64
IsSxsPassiveMode                                        int8
AVProductStatesIdentifier                              int64
AVProductsInstalled                                    int64
AVProductsEnabled                                      int64
CountryIdentifier                                      int64
CityIdentifier                                         int64
OrganizationIdentifier                                 int64
GeoNameIdentifier                                      int64
LocaleEnglishNameIdentifier                            int64
Platform                                               int16
Processor                                               int8
OsSuite                                                int64
OsPlatformSubRelease                                   int16
SkuEdition              